# **PROJECT2 : LA Airbnb Listing**


## **ETL : Extract, Transform, Load Project**

* <b>Extract:</b>   read the data, often from mupltiple sources/formats.<br>
* <b>Transform:</b>  clean and structure the data to suit business needs.<br>
* <b>Load:</b>  load the data into a database for storage that can be used for future analysis or business use.

### Data source:
* https://www.kaggle.com/oindrilasen/la-airbnb-listings


 <b>DATA EXTRACTION</b><br>
 
   <b>STEP 1: Import Modules And Data</b>

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os

In [ ]:
#Read the csv file
path = os.path.join("Resources", "Detail_listings.csv")
df=pd.read_csv(path)
df.head()

<b>STEP2. DATA EXPLORATION</b><br>
Explore Datasets to list Columns and their respective Data-types

In [ ]:
df.info()

<b>STEP3: DATA CLEANING<b>

In [ ]:
# Ensure each listing is a unique property
# len(df.listing_url.unique())
len(df)

In [ ]:
def price_cleaner(df, column):
    '''
    cleans str price by removing dollar sign, commas, and changing dtype to float
    
    df : dataframe to change
    column: selected column in quotes (eg. 'cost_column')
    '''
    df[column] = df[column].str.replace(',', '')
    df[column] = df[column].str.replace('$', '')
    df[column] = df[column].astype(float)

In [ ]:
#Establish baseline price column for comparisons
price_df = df[["id", "listing_url", "name", "price"]].copy().reset_index(drop=True)

#Change price to float (not str)
price_cleaner(price_df, "price")

price_df.head()

In [ ]:
#neighbourhood dataframe
neighborhood_df=df[["id","neighbourhood_cleansed","zipcode"]].dropna().reset_index(drop=True)
neighborhood_df = neighborhood_df.rename(columns={"neighbourhood_cleansed":"neighborhood"})
neighborhood_df

In [ ]:
# Merge to show comparison between neighborhood and price
neighborhood_price = pd.merge(
    neighborhood_df[["id","neighborhood"]]
    , price_df[["id","price"]]
    , on="id")
neighborhood_price.head()

In [ ]:
# host info data frame
host_df = df[[
    "id", "host_id","host_url", "host_name", "host_since", "host_is_superhost", "host_total_listings_count"
]].dropna().reset_index(drop=True)
host_df = host_df.rename(columns={"host_is_superhost":"superhost_status"})
host_df.head()

In [ ]:
# Merge to show comparison between superhost status and price
superhost_price = pd.merge(
    host_df[["host_id", "id", "superhost_status"]]
    , price_df[["id","price"]]
    , on="id"
)
superhost_price.head()

In [ ]:
# Property stats dataframe
property_stats_df = df[[
    "id","property_type", "room_type", "bedrooms","bathrooms"
]].dropna().reset_index(drop=True)
property_stats_df

In [ ]:
# Bedroom/price merged df
bedroom_price = pd.merge(
    property_stats_df[["id", "bedrooms"]]
    , price_df[["id","price"]]
    , on="id"
)
bedroom_price.head()

In [ ]:
#Bathroom/price merged df
bathroom_price = pd.merge(
    property_stats_df[["id", "bathrooms"]]
    , price_df[["id","price"]]
    , on="id"
)
bathroom_price.head()

In [ ]:
# Associated Fees dataframe
fees_df = df[[
    "id","security_deposit", "cleaning_fee","extra_people"
]].reset_index(drop=True)

price_cleaner(fees_df, "security_deposit")
price_cleaner(fees_df, "cleaning_fee")
price_cleaner(fees_df, "extra_people")

fees_df["security_deposit"]
fees_df["cleaning_fee"]
fees_df["extra_people"]

fees_df = fees_df.fillna(0)
fees_df.head()

In [ ]:
# Merge to show comparison between fees and price
fees_price = pd.merge(
    fees_df[["id", "security_deposit","cleaning_fee","extra_people"]]
    , price_df[["id","price"]]
    , on="id"
)
fees_price.head()

In [ ]:
# Property Reviews Dataframe
reviews_df = df[[
    "id","review_scores_rating", "review_scores_accuracy", "review_scores_cleanliness","review_scores_location","review_scores_value","reviews_per_month"
]].dropna().reset_index(drop=True)
reviews_df.columns = reviews_df.columns.str.replace('review_scores_', '')
reviews_df

In [ ]:
# Merge to show comparison between superhost status and price
reviews_df = pd.merge(
    reviews_df[["id", "rating","reviews_per_month"]]
    , price_df[["id","price"]]
    , on="id"
)
reviews_df.head()

<b>STEP4: DATA LOADING<b>